In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv("household_power_consumption.csv", parse_dates=['Date'], index_col='Date')

data.head()

C:\Users\Huzaifa\AppData\Local\Temp\ipykernel_11436\535478332.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv("household_power_consumption.csv", parse_dates=['Date'], index_col='Date')


,index,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date,,,,,,,,,
2007-01-01,0,0:00:00,2.58,0.136,241.97,10.6,0,0,0.0
2007-01-01,1,0:01:00,2.552,0.1,241.75,10.4,0,0,0.0
2007-01-01,2,0:02:00,2.55,0.1,241.64,10.4,0,0,0.0
2007-01-01,3,0:03:00,2.55,0.1,241.71,10.4,0,0,0.0
2007-01-01,4,0:04:00,2.554,0.1,241.98,10.4,0,0,0.0


In [4]:
data = pd.read_csv('household_power_consumption.csv', parse_dates=['Date'], index_col='Date')

data['Global_active_power'] = data['Global_active_power'].replace('?',np.nan)
data['Global_active_power'] = data['Global_active_power'].astype('float')
data = data.dropna(subset=['Global_active_power'])
data = data['Global_active_power'].values.reshape(-1, 1)

scalar = MinMaxScaler()
scaled = scalar.fit_transform(data)

C:\Users\Huzaifa\AppData\Local\Temp\ipykernel_11436\2778704427.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('household_power_consumption.csv', parse_dates=['Date'], index_col='Date')


In [8]:
def create_Sequence(data,seq_length):
    x,y = [],[]
    for i in range(len(data)-seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x),np.array(y)
        
seq_length = 24
X,y = create_Sequence(scaled,seq_length)

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
model = Sequential([
    Dense(64,activation='relu',input_shape = (seq_length,)),
    Dense(32,activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam',loss='mse')

c:\Users\Huzaifa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.0013 - val_loss: 7.9203e-04
Epoch 2/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 8.2473e-04 - val_loss: 7.7281e-04
Epoch 3/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 8.0946e-04 - val_loss: 7.4420e-04
Epoch 4/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 7.9080e-04 - val_loss: 7.5421e-04
Epoch 5/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 7.6764e-04 - val_loss: 7.3575e-04
Epoch 6/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 7.7355e-04 - val_loss: 7.2448e-04
Epoch 7/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 7.7177e-04 - val_loss: 7.2874e-04
Epoch 8/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 7.6891e-04 - val_loss: 7.0603e-04
Epoch 9/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 7.4693e-04 - val_loss: 7.7788e-04
Epoch 10/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 7.4724e-04 - val_loss: 7.0569e-04


In [20]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)

print(mean_absolute_error(y_test,y_pred))

1606/1606 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
0.010757267946608477


In [21]:
from sklearn.metrics import mean_squared_error
print("MSE:", mean_squared_error(y_test, y_pred))


MSE: 0.0007058651145593079


In [22]:
model.save('household_power_consumption.h5')